# Importing Libraries

In [102]:
import re
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score    

# Cleaning Data

- Data Cleaning and Preprocessing Workflow for News Articles


In [103]:
def clean_data(df):
    df = df.dropna(
        subset=["title", "content"]
    ).copy()  

    df["link"] = df["link"].fillna("No Link")
    df["id"] = df["id"].fillna(-1)
    df["content"] = df["content"].fillna("")

    df = df.drop_duplicates(subset=["title", "content"]).copy()

    df = df[
        df["content"].apply(len) > 50
    ].copy() 

    df = df.reset_index(drop=True)

    return df


file_path = "urdu_articles.csv"
df_urdu = pd.read_csv(file_path)

cleaned_df = clean_data(df_urdu)
df_urdu.columns

duplicates_count = cleaned_df.duplicated(subset=["title", "content"]).sum()
print("Number of duplicates:", duplicates_count)

min_length = cleaned_df["content"].apply(len).min()
print("Minimum content length:", min_length)
print("\n")

print(cleaned_df.head())

Number of duplicates: 0
Minimum content length: 92


   id                                              title  \
0   0  'باہوبلی 2' کے اداکار 47 سال کی عمر میں شادی ک...   
1   1  نیٹ فلکس سیریز تنازع : دھنش نے نینتھارا کے خلا...   
2   2  گلیڈی ایٹر 2' نے باکس آفس پر تہلکہ مچایا، 106 ...   
3   3  والد چنکی پانڈے طویل عرصے تک بےروزگار رہے، انن...   
4   4  رشی کپور نے فلم ’برفی‘ پر بیٹے رنبیر کو کیا مش...   

                                                link  \
0  https://www.express.pk/story/2735199/bahubali2...   
1  https://www.express.pk/story/2735191/netflixse...   
2  https://www.express.pk/story/2735188/gladiator...   
3  https://www.express.pk/story/2735173/chinkypan...   
4  https://www.express.pk/story/2735167/rishikapo...   

                                             content     gold_label  
0  بالی ووڈ اور ساؤتھ انڈین فلم انڈسٹری کے معروف ...  entertainment  
1  تامل فلم انڈسٹری کے دو مشہور ستاروں، دھنش اور ...  entertainment  
2  ہالی وڈ کی دو بڑی فلموں، میوزیکل ایڈ

## Tokenization and Bag of Words Analysis for Urdu News Articles

In [104]:
def tokenize_text(text):
    urdu_stop_words = set([
        "یہ", "میں", "ہے", "اور", "سے", "کہ", "کو", "پر", "نے", "کا", "کر", "کیا", "تو", 
        "کی", "ہم", "آپ", "جو", "کیوں", "کچھ", "وہ", "یہاں", "جہاں", "تھا", "جب", "تھے", 
        "بھی", "لیے", "ایک", "ہے", "نہیں", "صرف", "جیسا", "ان", "تم", "مجھے", "میرا", 
        "ہو", "چاہیے", "کس", "کن", "کون", "ہوں", "ہیں", "کے", "ا","اس","گیا","کہا","سی","کرنے","مطابق"
    ])
    
    tokens = re.findall(r"\w+", text.lower())

    return [word for word in tokens if word not in urdu_stop_words]


def create_bag_of_words(text_series):
    bow = Counter()
    for text in text_series:
        tokens = tokenize_text(text)
        bow.update(tokens)
    return bow

title_bow = create_bag_of_words(cleaned_df["title"])
bow = create_bag_of_words(cleaned_df["content"])

print("Top 20 words in titles:")
for word, freq in title_bow.most_common(20):
    print(f"{word}: {freq}")

print("\nTop 20 words in contents:")
for word, freq in bow.most_common(20):
    print(f"{word}: {freq}")

Top 20 words in titles:
پاکستان: 107
کیلئے: 81
پی: 68
اضافہ: 61
اے: 57
ٹرافی: 54
آئی: 46
ئی: 45
دیا: 42
خان: 41
بعد: 41
روپے: 41
ٹی: 40
اسرائیلی: 40
بھارتی: 37
دی: 37
بھارت: 37
اعلان: 35
2: 34
سال: 34

Top 20 words in contents:
پاکستان: 1199
جس: 869
بعد: 853
رہے: 794
ہوئے: 775
میڈیا: 754
ساتھ: 743
اپنے: 655
گا: 654
تک: 642
بھارتی: 630
دیا: 621
ہونے: 619
جانب: 619
روپے: 613
پی: 586
گئی: 575
انہوں: 566
اپنی: 565
ہی: 561


## Regularized Logistic Regression Model 
L1 Regularization

In [105]:
class RegularizedLogRegModel():
    def __init__(self, alpha, iterations):

        self.alpha = alpha
        self.iterations = iterations
        self.theta = 0
        self.bias = 0
        self.cost = []

    def sigmoid(self, x):

        z = 1/(1+np.exp(-x))
        return z
    
    def cross_entropy_loss(self, y_true, y_pred):

        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        loss = - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

        reg_l2 = (1 / (2*len(y_true))) * np.sum(self.theta**2) 
        reg_l1 = (self.alpha / (2*len(y_true))) * np.sum(np.abs(self.theta))

        return np.mean(loss) + reg_l1


    def fit(self, X_train1, y_train1):

        m,n = X_train1.shape
        self.theta = np.zeros(n)

        for i in range(self.iterations):

            z = np.dot(X_train1, self.theta) + self.bias
            h = self.sigmoid(z)

            gradient = (1/m) * (np.dot(X_train1.T, (h - y_train1)))
            bias_gradient = (1 / m) * np.sum(h - y_train1)

            self.theta -= self.alpha * gradient
            self.bias -= self.alpha * bias_gradient
            
            cost = self.cross_entropy_loss(y_train1, h)
            self.cost.append(cost)

        final_loss = self.cross_entropy_loss(y_train1, h)
        return final_loss

    def predict(self, X_test1):

        z = np.dot(X_test1, self.theta) + self.bias
        y_prob = self.sigmoid(z)
        y_pred = (y_prob >= 0.5).astype(int)    

        return y_prob, y_pred

    def evaluate(self, y_true, y_pred):
        
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')
        conf_matrix = confusion_matrix(y_true, y_pred)

        return {'accuracy': accuracy, 'f1_score': f1, 'confusion_matrix': conf_matrix}



## OvA (One vs All Logistic Classifier)

In [106]:
def create_feature_matrix(text_series, vocab):
    feature_matrix = []
    for text in text_series:
        tokens = tokenize_text(text)
        row = [tokens.count(word) for word in vocab]
        feature_matrix.append(row)
    return np.array(feature_matrix)

combined_text = cleaned_df["title"] + " " + cleaned_df["content"]
bow = create_bag_of_words(combined_text)
vocab = [word for word, _ in bow.most_common(1000)]

label_mapping = {label: idx for idx, label in enumerate(cleaned_df["gold_label"].unique())}
cleaned_df["gold_label"] = cleaned_df["gold_label"].map(label_mapping)

print("Label Mapping:", label_mapping)

X = create_feature_matrix(combined_text, vocab)  # Use the combined text as input
y = cleaned_df["gold_label"]
indices = cleaned_df.index  # Save original indices

# Train-Test Split with indices
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    X, y, indices, test_size=0.2, random_state=42
)

models = []
categories = sorted(np.unique(y))
for category in categories:
    print(f"Training classifier for category: {category}")
    y_binary = (y_train == category).astype(int) 
    model = RegularizedLogRegModel(alpha=0.1, iterations=1000)
    final_loss = model.fit(X_train, y_binary)
    print(f"Final loss for category {category}: {final_loss}")
    models.append(model)

probs = []
predictions = []
for model in models:
    y_prob, y_pred = model.predict(X_test)
    probs.append(y_prob)
    predictions.append(y_pred)

probs = np.array(probs).T 
final_predictions = np.argmax(probs, axis=1).astype(int) 

accuracy = accuracy_score(y_test, final_predictions)
f1 = f1_score(y_test, final_predictions, average='weighted')
conf_matrix = confusion_matrix(y_test, final_predictions)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)


Label Mapping: {'entertainment': 0, 'business': 1, 'sports': 2, 'science-technology': 3, 'world': 4}
Training classifier for category: 0
Final loss for category 0: 0.01489320215243476
Training classifier for category: 1
Final loss for category 1: 0.012979927705214458
Training classifier for category: 2
Final loss for category 2: 0.014559640359764214
Training classifier for category: 3
Final loss for category 3: 0.01894340798801794
Training classifier for category: 4
Final loss for category 4: 0.0227471848039124
Accuracy: 0.956140350877193
F1 Score: 0.9558654475129876
Confusion Matrix:
[[42  0  0  0  1]
 [ 0 47  0  4  3]
 [ 0  0 34  0  0]
 [ 0  0  0 52  0]
 [ 1  0  0  1 43]]


### Final Misclassifications

In [107]:
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

test_df = cleaned_df.iloc[test_indices].copy()

test_df['gold_label'] = test_df['gold_label'].map(reverse_label_mapping)
test_df['predicted_label'] = [reverse_label_mapping[int(label)] for label in final_predictions]

test_df['is_misclassified'] = test_df['gold_label'] != test_df['predicted_label']

misclassified_df = test_df[test_df['is_misclassified']]

misclassified_count = len(misclassified_df)
print(f"Total Misclassified Samples: {misclassified_count}")

print("\nMisclassified Samples (Debugging):")
print(misclassified_df[['gold_label', 'predicted_label']].head(10))

print("\nMisclassified Samples (Detailed):")
for idx, row in misclassified_df.head(10).iterrows():
    print(f"Index: {idx}")
    print(f"Actual Label: {row['gold_label']}")
    print(f"Predicted Label: {row['predicted_label']}")
    print(f"Content: {row['content']}")
    print("-" * 50)


Total Misclassified Samples: 10

Misclassified Samples (Debugging):
         gold_label     predicted_label
428        business  science-technology
113        business  science-technology
359   entertainment               world
429        business               world
634           world  science-technology
767        business               world
439        business  science-technology
1013          world       entertainment
832        business               world
66         business  science-technology

Misclassified Samples (Detailed):
Index: 428
Actual Label: business
Predicted Label: science-technology
Content: کراچی میں جاری دفاعی نمائش آئیڈیاز میں پاکستان نے پہلی مرتبہ اپنا تیار کردہ لانگ رینج ائیر سرویلنس ریڈار سسٹم نمائش کیلئے پیش کردیا۔ ڈائریکٹر بلیو سرچ پرائیویٹ لمیٹڈ اویس رؤف کا کہنا تھا کہ ڈرونز اور انتہائی تیز رفتار میزائل سسٹمز نے دور حاضر میں جنگوں کے طریقہ کار کو بدل کر رکھ دیا ہے، روس اور یوکرین کی جنگ ہو یا دنیا میں جاری کوئی اور  تنازعہ، دنیا دیکھ رہی ہے کہ فوجی نقل و

### Final Correct Classifications

In [108]:
test_df['is_correctly_classified'] = test_df['gold_label'] == test_df['predicted_label']

correctly_classified_df = test_df[test_df['is_correctly_classified']]

correctly_classified_count = len(correctly_classified_df)
print(f"Total Correctly Classified Samples: {correctly_classified_count}")

print("\nCorrectly Classified Samples:")
for idx, row in correctly_classified_df.head(10).iterrows():
    print(f"Index: {idx}")
    print(f"Actual Label: {row['gold_label']}")
    print(f"Predicted Label: {row['predicted_label']}")
    print(f"Content: {row['content']}")
    print("-" * 50)


Total Correctly Classified Samples: 218

Correctly Classified Samples:
Index: 786
Actual Label: business
Predicted Label: business
Content: اسٹیٹ بینک آف پاکستان نے کہا ہے کہ اکتوبر میں 35 کروڑ ڈالرز کا بڑا سر پلس حاصل ہوا ہے۔ جاری کیے گئے اعلامیے میں اسٹیٹ بینک نے کہا ہے کہ اکتوبر میں ترسیلاتِ زر ماہانہ لحاظ سے 7 فیصد اور سالانہ حساب سے 24 فیصد بڑھی ہیں۔ پاکستان کا اکتوبر میں مسلسل تیسرے ماہ کرنٹ اکاؤنٹ سرپلس رہا، اکتوبر میں پاکستان کا کرنٹ اکاؤنٹ 35 کروڑ ڈالرز سرپلس رہا۔ پاکستان اسٹاک ایکسچینج میں آج کاروباری ہفتے کا مثبت آغاز ہوا، جو بعد میں ملے جلے رجحان میں تبدیل ہونے کے بعد پھر مثبت ہو گیا۔ اسٹیٹ بینک نے کہا کہ اکتوبر میں برآمدات 3 ارب 70 کروڑ ڈالرز رہیں جبکہ درآمدات 5 ارب 50 کروڑ ڈالرز رہیں۔ بینک کا کہنا ہے کہ اکتوبر میں بیرون ممالک سے ترسیلات زر 3 ارب 5 کروڑ ڈالرز رہیں۔
--------------------------------------------------
Index: 902
Actual Label: entertainment
Predicted Label: entertainment
Content: جنوبی بھارت کی تامل فلموں کے معروف اداکار دھنوش نے بدھ کو اپنے سوشل میڈیا ہین